In [1]:
from dotenv import load_dotenv
import os
from datetime import datetime, timezone, timedelta, tzinfo
import pandas as pd
import fluxy as fl
from itertools import product

from analysis.analysis_functions import electric_variables, get_data, kwh_meters_rename_dict, electrical_groups_dict_inv, electrical_consumption_groups_dict_inv, electrical_consumption_groups

%load_ext autoreload 
%autoreload 2

load_dotenv()

API_TOKEN = os.getenv("ANALYSIS_API_TOKEN")

In [2]:
start = datetime(2024,8,22,  tzinfo = timezone.utc) - timedelta(hours = 3)
stop = datetime(2024,10,22,  tzinfo = timezone.utc) - timedelta(hours = 3)
window_size = timedelta(days = 1)

In [5]:
electric_data = await get_data([{"_field": variable} for variable in electric_variables], (start, stop), window_size, fl.WindowOperation.MEAN, API_TOKEN)

electric_data.index = electric_data.index.tz_convert('Europe/Madrid')
electric_data = electric_data.rename(columns = kwh_meters_rename_dict) # fix wrong naming in databases

kwh_daily = electric_data.T.groupby({**electrical_consumption_groups_dict_inv, **electrical_groups_dict_inv}).sum().T.rename(columns = {'pv_power':'pv','grid_power':'grid'})*24/1000
functional_consumption_groups = {"office":"office", "workshop":"workshop","simulator":"simulator","kitchen_sanitary_1":"sanitary","kitchen_sanitary_2":"catering","center_1":"catering", "center_2":"events", "power_hub":"power_hub", "supply_box":"power_hub"}
groups = kwh_daily.T.groupby(functional_consumption_groups).sum().T
kwh_daily = kwh_daily[['pv','grid']].join(groups).assign(total_consumed=groups.sum(1))
electric_csv_data = kwh_daily.rename(columns = {col:col+'_kwh' for col in kwh_daily.columns})

In [37]:
volume_variables = [
    "fresh_to_kitchen_flow_sensor_total_volume",
    "technical_to_wash_off_flow_sensor_total_volume",
    "technical_to_sanitary_flow_sensor_total_volume"
]

volumes_data = await get_data([{"_field": variable} for variable in volume_variables], (start, stop), window_size, fl.WindowOperation.LAST, API_TOKEN)
volumes_data.index = volumes_data.index.tz_convert('Europe/Madrid')

daily_liters = volumes_data.diff().rename(columns = {"fresh_to_kitchen_flow_sensor_total_volume":"kitchen_water_l",
    "technical_to_wash_off_flow_sensor_total_volume":"wash_off_water_l",
    "technical_to_sanitary_flow_sensor_total_volume":"sanitary_water_l"})*1000


In [38]:
csv_data = electric_csv_data.join(daily_liters)

In [58]:
csv_data.to_csv('powerhub_daily_electric_water.csv')